In [8]:
#! /bin/env python
# -*- coding: utf-8 -*-
"""
预处理
corpus其实已经非常好了，已经整理成[label, text]的形式
我们需要做的就是，将text分词,去停用词
"""
from tools.tokenizer.wordCut import WordCut


word_divider = WordCut()
file_path = '/home/zhouchengyu/haiNan/textClassifier/data/cnews/cnews.val.txt'
write_path = '/home/zhouchengyu/haiNan/textClassifier/data/cnews/val_token.txt'
with open(write_path, 'w') as w:
    with open(file_path, 'r') as f:
        for line in f.readlines():
            line = line.decode('utf-8').strip()
            token_sen = word_divider.seg_sentence(line.split('\t')[1])
            w.write(line.split('\t')[0].encode('utf-8') + '\t' + token_sen.encode('utf-8') + '\n')    

In [10]:
"""
val数据集最短...所有调通它的分词后
我们接下来就要考虑，将train和test文件也分词处理
保存各自的token文件到本地

多进程处理
"""
import multiprocessing


tmp_catalog = '/home/zhouchengyu/haiNan/textClassifier/data/cnews/'
file_list = [tmp_catalog+'cnews.train.txt', tmp_catalog+'cnews.test.txt']
write_list = [tmp_catalog+'train_token.txt', tmp_catalog+'test_token.txt']

def tokenFile(file_path, write_path):
    word_divider = WordCut()
    with open(write_path, 'w') as w:
        with open(file_path, 'r') as f:
            for line in f.readlines():
                line = line.decode('utf-8').strip()
                token_sen = word_divider.seg_sentence(line.split('\t')[1])
                w.write(line.split('\t')[0].encode('utf-8') + '\t' + token_sen.encode('utf-8') + '\n') 
    print file_path + ' has been token and token_file_name is ' + write_path

pool = multiprocessing.Pool(processes=4)
for file_path, write_path in zip(file_list, write_list):
    pool.apply_async(tokenFile, (file_path, write_path, ))
pool.close()
pool.join() # 调用join()之前必须先调用close()
print "Sub-process(es) done."

/home/zhouchengyu/haiNan/textClassifier/data/cnews/cnews.test.txt has been token and token_file_name is /home/zhouchengyu/haiNan/textClassifier/data/cnews/test_token.txt
/home/zhouchengyu/haiNan/textClassifier/data/cnews/cnews.train.txt has been token and token_file_name is /home/zhouchengyu/haiNan/textClassifier/data/cnews/train_token.txt
Sub-process(es) done.


In [6]:
"""
多进程速度还是不错的，但是跟木板效应一样，
最终的执行总时间，还是跟最大的文件有关。

"""

def constructDataset(path):
    """
    path: file path
    rtype: lable_list and corpus_list
    """
    label_list = []
    corpus_list = []
    with open(path, 'r') as p:
        for line in p.readlines():
            label_list.append(line.split('\t')[0])
            corpus_list.append(line.split('\t')[1])
    return label_list, corpus_list

tmp_catalog = '/home/zhouchengyu/haiNan/textClassifier/data/cnews/'
file_path = 'val_token.txt'
val_label, val_set = constructDataset(tmp_catalog+file_path)
print len(val_set)

5000


In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


tmp_catalog = '/home/zhouchengyu/haiNan/textClassifier/data/cnews/'
write_list = [tmp_catalog+'train_token.txt', tmp_catalog+'test_token.txt']

tarin_label, train_set = constructDataset(write_list[0]) # 50000
test_label, test_set = constructDataset(write_list[1]) # 10000
# 计算tf-idf
corpus_set = train_set + val_set + test_set # 全量计算tf-idf
print "length of corpus is: " + str(len(corpus_set))
vectorizer = CountVectorizer(min_df=1e-5) # drop df < 1e-5,去低频词
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(vectorizer.fit_transform(corpus_set))
words = vectorizer.get_feature_names()
print "how many words: {0}".format(len(words))
print "tf-idf shape: ({0},{1})".format(tfidf.shape[0], tfidf.shape[1])

length of corpus is: 65000
how many words: 379000
tf-idf shape: (65000,379000)


In [12]:
"""
上面有一个容易忽略的点：
计算tf-idf需要将train, val, test三方面的数据集全部计算，这样提取到的特征才更加准确
"""
from sklearn import preprocessing

# encode label
corpus_label = tarin_label + val_label + test_label
encoder = preprocessing.LabelEncoder()
corpus_encode_label = encoder.fit_transform(corpus_label)
train_label = corpus_encode_label[:50000]
val_label = corpus_encode_label[50000:55000]
test_label = corpus_encode_label[55000:]
# get tf-idf dataset
train_set = tfidf[:50000]
val_set = tfidf[50000:55000]
test_set = tfidf[55000:]

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# LogisticRegression classiy model
lr_model = LogisticRegression()
lr_model.fit(train_set, train_label)
print "val mean accuracy: {0}".format(lr_model.score(val_set, val_label))
y_pred = lr_model.predict(test_set)
print classification_report(test_label, y_pred)

mean accuracy: 0.9626
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1000
          1       0.99      0.98      0.98      1000
          2       0.94      0.87      0.91      1000
          3       0.91      0.91      0.91      1000
          4       0.97      0.93      0.95      1000
          5       0.97      0.98      0.98      1000
          6       0.93      0.96      0.95      1000
          7       0.99      0.97      0.98      1000
          8       0.94      0.99      0.96      1000
          9       0.95      0.99      0.97      1000

avg / total       0.96      0.96      0.96     10000



In [14]:
# 随机森林分类器
from sklearn.ensemble import RandomForestClassifier    


rf_model = RandomForestClassifier(n_estimators=200, random_state=1080)
rf_model.fit(train_set, train_label)
print "val mean accuracy: {0}".format(rf_model.score(val_set, val_label))
y_pred = rf_model.predict(test_set)
print classification_report(test_label, y_pred)

val mean accuracy: 0.9228
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1000
          1       0.98      0.98      0.98      1000
          2       0.89      0.57      0.69      1000
          3       0.81      0.97      0.88      1000
          4       0.95      0.89      0.92      1000
          5       0.97      0.96      0.97      1000
          6       0.85      0.94      0.89      1000
          7       0.95      0.97      0.96      1000
          8       0.95      0.97      0.96      1000
          9       0.91      0.99      0.95      1000

avg / total       0.93      0.92      0.92     10000



In [ ]:
"""
上面采用逻辑回归分类器和随机森林分类器做对比：
可以发现，除了个别分类随机森林方法有较大进步，大部分都差于逻辑回归分类器
并且200棵树的随机森林耗时过长，比起逻辑回归分类器来说，运算效率太低
"""
